# Airbnb Capstone Project

## London listings.csv test

In [5]:
import pandas as pd
import numpy as np
from py_functions import increase_bbox
import requests
import json

In [8]:
# London
# Define path, .gz archive file name, country and city for url
path ='data/'
gz_file = "listings.csv.gz"
country = "united-kingdom"
state = "england"
city = "london"
url = f"http://data.insideairbnb.com/{country}/{state}/{city}/2023-03-14/data/{gz_file}"

In [9]:
# Create new directory for city
!mkdir {path}{city}

mkdir: data/london: File exists


In [10]:
# Download the .gz file
r = requests.get(url)
with open(path+city+'/'+gz_file, 'wb') as f:
    f.write(r.content)

In [11]:
# Unzip the .gz file and save the content as pd.DataFrame via read_csv
with gzip.open(path+city+'/'+gz_file) as f:

    listings = pd.read_csv(f)

In [12]:
listings.head(3)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,714569379355913481,https://www.airbnb.com/rooms/714569379355913481,20230314070633,2023-03-14,city scrape,Lovely private bedroom in Muswell Hill.,Take a break and unwind at this peaceful oasis.,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,39009854,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
1,808038970516277767,https://www.airbnb.com/rooms/808038970516277767,20230314070633,2023-03-14,city scrape,Studio Flat Franklin London,Brand New Modern Kitchen<br />Close to Excelle...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,495977998,...,NaN,NaN,NaN,NaN,t,7,7,0,0,NaN
2,822557738577472503,https://www.airbnb.com/rooms/822557738577472503,20230314070633,2023-03-14,city scrape,PropertyPlug - 2Bed Flat in Edgware SmartTV WiFi,Enjoy easy access to everything from this perf...,NaN,https://a0.muscache.com/pictures/d77957d5-695a...,325629338,...,NaN,NaN,NaN,NaN,t,4,4,0,0,NaN


In [13]:
# Outermost longitude/latitude points of the Airbnb listings
listings.describe()[["latitude", "longitude"]].loc[["min", "max"]]

,latitude,longitude
min,51.295937,-0.49780
max,51.681142,0.28857


(northern hemisphere)

latitude max = north

latitude min = south

longitude max = east

longitude min = west

Increasing the maxs by 0.01 and decreasing the mins by 0.01 will shift the outline's border by a bit more than 1km in each direction.

See increase_bbox function in py_functions.py

requests and json method:

In [8]:
import requests
import json
from py_functions import increase_bbox

In [9]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
(node["amenity"~"^(bar|pub|restaurant)$"](51.2867602,-0.5103751,51.6918741,0.3340155);
);
out body;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [10]:
data_norm = pd.json_normalize(data, record_path="elements")

In [11]:
pd.set_option('display.max_columns', None)
data_norm.head()

,type,id,lat,lon,tags.addr:city,tags.addr:housename,tags.addr:housenumber,tags.addr:postcode,tags.addr:street,tags.amenity,tags.cuisine,tags.diet:gluten_free,tags.diet:meat,tags.diet:vegan,tags.diet:vegetarian,tags.food,tags.internet_access,tags.name,tags.opening_hours:covid19,tags.operator,tags.outdoor_seating,tags.payment:american_express,tags.payment:contactless,tags.payment:credit_cards,tags.payment:debit_cards,tags.payment:maestro,tags.payment:mastercard,tags.payment:visa,tags.smoking,tags.toilets,tags.toilets:access,tags.real_ale,tags.addr:country,tags.brand,tags.brand:wikidata,tags.brand:wikipedia,tags.indoor_seating,tags.website,tags.wikidata,tags.fhrs:id,tags.fhrs:local_authority_id,tags.source:addr,tags.wheelchair,tags.source,tags.upload_tag,tags.ale,tags.brewery,tags.check_date,tags.email,tags.opening_hours,tags.phone,tags.real_fire,tags.created_by,tags.note,tags.wheelchair:description,tags.dog,tags.contact:email,tags.contact:phone,tags.wikipedia,tags.old_name,tags.contact:website,tags.opening_hours:food,tags.foot,tags.fhrs:authority,tags.fhrs:inspectiondate,tags.fhrs:rating,tags.addr:suburb,tags.former_name,tags.previous_name,tags.level,tags.takeaway,tags.fhrs:confidence_management,tags.fhrs:hygiene,tags.fhrs:structural,tags.addr:village,tags.survey:date,tags.alt_name,tags.historic,tags.contact:facebook,tags.contact:instagram,tags.contact:twitter,tags.lgbtq,tags.opening_hours:kitchen,tags.website:menu,tags.source:name,tags.opening_hours:signed,tags.air_conditioning,tags.bar,tags.postal_code,tags.description,tags.beer_garden,tags.fixme:place,tags.facebook,tags.twitter,tags.internet_access:fee,tags.source:postcode,tags.source:addr:postcode,tags.floor,tags.toilets:wheelchair,tags.start_date,tags.is_in,tags.source:old_name,tags.eat_in,tags.fhrs:name,tags.addr:unit,tags.not:addr:housenumber,tags.not:addr:postcode,tags.source:not:addr,tags.source:not:addr:housenumber,tags.dontimport:fhrs:addrline1,tags.dontimport:fhrs:addrline2,tags.dontimport:fhrs:businesstype,tags.microbrewery,tags.fixme,tags.addr:parentstreet,tags.not:addr:street,tags.addr:city:fa,tags.toilets:female,tags.toilets:male,tags.toilets:unisex,tags.wikimedia_commons,tags.real_cider,tags.fixme:addr:1,tags.accommodation,tags.comedy,tags.description:floor,tags.note:beer,tags.venue,tags.loc_name,tags.photo,tags.addr:interpolation,tags.roof_terrace,tags.opening_hours:url,tags.disused,tags.atm,tags.check_date:opening_hours,tags.sport,tags.tourism,tags.closed,tags.addr:floor,tags.wifi,tags.craft_keg,tags.warning,tags.reservation,tags.capacity,tags.currency:XLT,tags.payment:cash:XLT-BXTP,tags.payment:text:XLT-BXTP,tags.image,tags.garden,tags.addr:town,tags.payment:cash,tags.gay,tags.branch,tags.amenity_1,tags.internet_access:ssid,tags.payment:bancomat,tags.payment:visa_debit,tags.payment:visa_electron,tags.url,tags.old_amenity,tags.short_name,tags.wine,tags.sky,tags.layer,tags.delivery,tags.instagram,tags.deli,tags.drive_in,tags.shop,tags.note:name:ko,tags.contact:untappd,tags.min_age,tags.drive_through,tags.name:ja,tags.name:en,tags.owner,tags.cocktails,tags.name:zh,tags.music_genre,tags.old_cuisine,tags.ref:vatin,tags.not:name,tags.diet:pescetarian,tags.seats,tags.operator:wikidata,tags.type,tags.football,tags.internet_access:free,tags.disused:name,tags.addr:place,tags.floor:material,tags.ref,tags.level:ref,tags.currency:XBT,tags.contact:fax,tags.theme,tags.was:amenity,tags.alcohol,tags.FIXME,tags.building,tags.old_fhrs:id,tags.old_fhrs:local_authority_id,tags.check_date:diet:vegetarian,tags.name:fa,tags.note:name,tags.quiz,tags.name:lit,tags.addr:county,tags.changing_table,tags.changing_table:location,tags.currency:GBP,tags.payment:lightning,tags.payment:lightning_contactless,tags.payment:onchain,tags.breakfast,tags.trendy,tags.addr:substreet,tags.fax,tags.addr:flats,tags.old_name:zh,tags.addr:flat,tags.last_check,tags.addr:full,tags.number,tags.diet:dairy_free,tags.outdoor_seating:comfort,tags.name:es,tags.name:gl,tags.diet:kosher,tags.disused:shop,tags.source:

In [12]:
data_norm[["tags.name", "tags.amenity", "tags.cuisine"]]

,tags.name,tags.amenity,tags.cuisine
0,King of Prussia,pub,pizza;burger
1,Central Restaurant,restaurant,NaN
2,The Catcher in the Rye,pub,NaN
3,The Tally Ho,pub,NaN
4,Railway Bell,pub,NaN
...,...,...,...
7754,NaN,bar,NaN
7755,The Tranquil Tiger,pub,NaN
7756,Firecracker,restaurant,oriental
7757,The Coriander,restaurant,indian


In [13]:
data_norm.shape

(7759, 384)

## Lisbon pipeline test

In [1]:
import gzip

In [2]:
# Define path, .gz archive file name, country and city for url
path ='data/'
gz_file = "listings.csv.gz"
country = "portugal"
state = "lisbon"
city = "lisbon"
url = f"http://data.insideairbnb.com/{country}/{state}/{city}/2023-03-19/data/{gz_file}"

In [3]:
# Create new directory for city
!mkdir {path}{city}

In [6]:
# Download the .gz file
r = requests.get(url)
with open(path+city+'/'+gz_file, 'wb') as f:
    f.write(r.content)

In [7]:
# Unzip the .gz file and save the content as pd.DataFrame via read_csv
with gzip.open(path+city+'/'+gz_file) as f:

    listings = pd.read_csv(f)

In [17]:
# Increase outside border of listings
lisbon_bbox = increase_bbox(listings)

In [18]:
lisbon_bbox

{'north_shifted': 39.312129999999996,
 'south_shifted': 38.66715,
 'west_shifted': -9.49789,
 'east_shifted': -8.85285}

In [19]:
# Get OSM data for slightly bigger bbox
# Example for nodes with an entry for "cuisine"
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = f"""
[out:json];
(node["cuisine"]({lisbon_bbox["south_shifted"]},{lisbon_bbox["west_shifted"]},{lisbon_bbox["north_shifted"]},{lisbon_bbox["east_shifted"]});
);
out body;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

data_norm = pd.json_normalize(data, record_path="elements")

In [20]:
data_norm.shape

(2041, 181)

In [21]:
data_norm.head()

,type,id,lat,lon,tags.addr:city,tags.addr:housenumber,tags.addr:postcode,tags.addr:street,tags.addr:unit,tags.amenity,tags.cuisine,tags.name,tags.phone,tags.opening_hours,tags.created_by,tags.email,tags.name:en,tags.name:pt,tags.website,tags.fixme,tags.phone_1,tags.addr:country,tags.addr:place,tags.internet_access,tags.name:zh,tags.operator,tags.shop,tags.start_date,tags.toilets:wheelchair,tags.tourism,tags.website:menu,tags.wheelchair,tags.wikipedia,tags.source,tags.brand,tags.brand:wikidata,tags.official_name,tags.takeaway,tags.wikidata,tags.brand:wikipedia,tags.amenity_1,tags.amenity_2,tags.internet,tags.internet_access:fee,tags.internet_access:ssid,tags.outdoor_seating,tags.smoking,tags.branch,tags.addr:housename,tags.contact:facebook,tags.capacity,tags.contact:mobile,tags.bicycle,tags.cycleway,tags.maxspeed,tags.surface,tags.description,tags.wifi,tags.contact:phone,tags.alt_name,tags.diet:vegetarian,tags.int_name,tags.old_name,tags.contact:email,tags.contact:website,tags.check_date,tags.addr:floor,tags.drive_through,tags.level,tags.lunch,tags.contact:instagram,tags.facebook,tags.check_date:currency:XBT,tags.currency:XBT,tags.payment:lightning,tags.payment:lightning_contactless,tags.payment:onchain,tags.wheelchair:description,tags.delivery,tags.sustenance,tags.payment:cash,tags.name:ru,tags.image,tags.mapillary,tags.survey:date,tags.theme,tags.music_genre,tags.addr:door,tags.payment:american_express,tags.payment:mastercard,tags.payment:notes,tags.payment:visa_debit,tags.payment:visa_electron,tags.building,tags.note,tags.bar,tags.payment:visa,tags.diet:halal,tags.contact:twitter,tags.breakfast,tags.air_conditioning,tags.phone_2,tags.addr:housenumber_1,tags.disused:amenity,tags.opening_hours:signed,tags.drive_in,tags.microbrewery,tags.organic,tags.payment:credit_cards,tags.payment:cryptocurrencies,tags.payment:debit_cards,tags.payment:electronic_purses,tags.payment:cards,tags.building:levels,tags.fax,tags.historic,tags.addr:province,tags.diet:vegan,tags.nif:pt,tags.changing_table,tags.fuel,tags.atm,tags.craft,tags.name:es,tags.ref:vatin,tags.name:fr,tags.fast_food,tags.cafe,tags.payment:contactless,tags.name:nl,tags.name:de,tags.check_date:opening_hours,tags.changing_table:fee,tags.highchair,tags.mobile,tags.toilets,tags.toilets:access,tags.drink:wine,tags.indoor_seating,tags.contact:mobile_1,tags.serving_system,tags.diet:meat,tags.brewery,tags.cash,tags.drink,tags.access,tags.diet:chicken,tags.opening_hours:covid19,tags.payment:maestro,tags.contact:phone_1,tags.contact:phone_2,tags.drink:beer,tags.diet:gluten_free,tags.service:bicycle:rental,tags.service:bicycle:repair,tags.service:bicycle:retail,tags.payment:coins,tags.disused:bar,tags.reservation,tags.name:ja,tags.diet:mediterranean,tags.diet:organic,tags.diet:omnivore,tags.diet:pescetarian,tags.phone:mobile,tags.payment:card,tags.diet:healthy,tags.diet:non-vegetarian,tags.diet:lactose_free,tags.short_name,tags.drink:cocktails,tags.drink:infusion,tags.natural,tags.payment:app,tags.stars,tags.alt_name:en,tags.brand:website,tags.brand:short,tags.layer,tags.takeaway:phone,tags.diet:fish
0,node,321604349,38.689011,-9.353835,Parede,13,2775-271,Rua Capitão Leitão,b,cafe,brasilian;snacks,Fénix Café,+351 963 348 481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,327373366,38.999764,-9.416248,Ribamar,NaN,NaN,EN 247,NaN,restaurant,seafood,Ribamariscos,+351 915 9

In [22]:
data_norm[["tags.name","tags.cuisine","tags.amenity"]]

,tags.name,tags.cuisine,tags.amenity
0,Fénix Café,brasilian;snacks,cafe
1,Ribamariscos,seafood,restaurant
2,Terra Mar,fish;portuguese;seafood,restaurant
3,Bom Amigo,portuguese;asian,restaurant
4,Pastéis de Belém,regional,NaN
...,...,...,...
2036,Sushi Mish Mish,japanese,restaurant
2037,L'Ami Malo,french,cafe
2038,Italian Republic,italian,restaurant
2039,Churrasqueira de Alvalade,steak_house,restaurant


Web scraping test

In [5]:
from bs4 import BeautifulSoup

In [8]:
url = "http://insideairbnb.com/get-the-data/"
page = requests.get(url)

In [9]:
soup = BeautifulSoup(page.content, "html.parser")

In [10]:
print(soup)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=0.75, maximum-scale=0.75" name="viewport"/><meta content="Gatsby 4.25.4" name="generator"/><meta data-react-helmet="true" href="favicon.ico" name="icon"/><meta content="Adding data to the debate" data-react-helmet="true" name="description"/><meta content="Get the Data" data-react-helmet="true" property="og:title"/><meta content="Adding data to the debate" data-react-helmet="true" property="og:description"/><meta content="website" data-react-helmet="true" property="og:type"/><meta content="summary" data-react-helmet="true" name="twitter:card"/><meta content="@Murray" data-react-helmet="true" name="twitter:creator"/><meta content="Get the Data" data-react-helmet="true" name="twitter:title"/><meta content="Adding data to the debate" data-react-helmet="true" name="twitter:description"/><style data-href="/styles.0762ab8b9dcfa2654